In [ ]:
## Analysis plan
Load `measurements/compute_20251219_123128.csv` in chunks, compute predictor `delta = IMEAS - X1`, filter measurements to 0.1–100 nA for each dependent (`OUT1_current`, `OUT2_current`), fit a linear model (numpy polyfit), report slope/intercept/R^2/pearson r, and save scatter+fit plots to `analysis_plots/`.


In [1]:
# Robust chunked analysis: filter to [0.1, 100] nA and fit linear models using numpy
import csv
from pathlib import Path
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import traceback

csv_path = Path(r"c:\Columbia\AdamResearch\kalman_2025\measurements\compute_20251219_123128.csv")
plot_dir = Path('analysis_plots')
plot_dir.mkdir(exist_ok=True)

lo_nA = 0.1
hi_nA = 100.0

# Accumulators for each dependent variable
x_out1 = []
y_out1 = []
x_out2 = []
y_out2 = []

try:
    # Use pandas in chunked mode if available, otherwise fallback to csv.reader
    try:
        import pandas as pd
        it = pd.read_csv(csv_path, usecols=['X1','IMEAS','OUT1_current','OUT2_current'], chunksize=20000)
        for chunk in it:
            delta = (chunk['IMEAS'] - chunk['X1']).to_numpy()
            out1_n = (chunk['OUT1_current'] * 1e9).to_numpy()
            out2_n = (chunk['OUT2_current'] * 1e9).to_numpy()
            # mask for valid numeric values
            mask1 = np.isfinite(delta) & np.isfinite(out1_n) & (out1_n >= lo_nA) & (out1_n <= hi_nA)
            mask2 = np.isfinite(delta) & np.isfinite(out2_n) & (out2_n >= lo_nA) & (out2_n <= hi_nA)
            x_out1.extend(delta[mask1].tolist())
            y_out1.extend(out1_n[mask1].tolist())
            x_out2.extend(delta[mask2].tolist())
            y_out2.extend(out2_n[mask2].tolist())
    except Exception as e:
        # fallback
        print('Pandas chunking failed, falling back to csv.reader:', e)
        with open(csv_path, newline='') as fh:
            reader = csv.DictReader(fh)
            for row in reader:
                try:
                    delta = float(row['IMEAS']) - float(row['X1'])
                    out1_n = float(row['OUT1_current']) * 1e9
                    out2_n = float(row['OUT2_current']) * 1e9
                except Exception:
                    continue
                if lo_nA <= out1_n <= hi_nA:
                    x_out1.append(delta); y_out1.append(out1_n)
                if lo_nA <= out2_n <= hi_nA:
                    x_out2.append(delta); y_out2.append(out2_n)

    def fit_and_plot(x, y, name):
        x = np.asarray(x)
        y = np.asarray(y)
        n = len(x)
        if n < 3:
            print(f"{name}: only {n} points in [{lo_nA}, {hi_nA}] nA — not enough to fit")
            return {'n': n}
        # fit
        slope, intercept = np.polyfit(x, y, 1)
        yhat = intercept + slope * x
        ss_res = np.sum((y - yhat)**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        r2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan
        # pearson
        pearson_r = np.corrcoef(x, y)[0,1]
        # save plot
        xs = np.linspace(x.min(), x.max(), 200)
        ys = intercept + slope * xs
        plt.figure(figsize=(6,4))
        plt.scatter(x, y, s=8)
        plt.plot(xs, ys, color='C1')
        plt.xlabel('IMEAS - X1 (units in file)')
        plt.ylabel(f'{name} (nA)')
        plt.title(f'{name} vs delta (n={n})\nR^2={r2:.4f}, slope={slope:.4e} nA/unit')
        plt.grid(alpha=0.3)
        p = plot_dir / f'{name}_vs_delta.png'
        plt.savefig(p, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"{name}: n={n}, slope={slope:.4e} nA/unit, intercept={intercept:.4e} nA, R^2={r2:.4f}, pearson_r={pearson_r:.4f}, plot={p}")
        return dict(n=n, slope=slope, intercept=intercept, r2=r2, pearson_r=pearson_r, plot=str(p))

    res1 = fit_and_plot(x_out1, y_out1, 'OUT1_current')
    res2 = fit_and_plot(x_out2, y_out2, 'OUT2_current')
    print('\nDone. Results:')
    print('OUT1:', res1)
    print('OUT2:', res2)

except Exception:
    print('EXCEPTION during analysis:')
    traceback.print_exc()


OUT1_current: n=108067, slope=6.9760e+06 nA/unit, intercept=1.3363e+01 nA, R^2=0.0000, pearson_r=0.0045, plot=analysis_plots\OUT1_current_vs_delta.png
OUT2_current: n=114461, slope=5.5505e+07 nA/unit, intercept=1.2179e+01 nA, R^2=0.0022, pearson_r=0.0472, plot=analysis_plots\OUT2_current_vs_delta.png

Done. Results:
OUT1: {'n': 108067, 'slope': np.float64(6975985.988298406), 'intercept': np.float64(13.363148783276436), 'r2': np.float64(2.0445506327959606e-05), 'pearson_r': np.float64(0.004521670745176656), 'plot': 'analysis_plots\\OUT1_current_vs_delta.png'}
OUT2: {'n': 114461, 'slope': np.float64(55504929.65051203), 'intercept': np.float64(12.179113234412194), 'r2': np.float64(0.0022286920980558733), 'pearson_r': np.float64(0.047209025599518055), 'plot': 'analysis_plots\\OUT2_current_vs_delta.png'}


In [2]:
# Search for a 'mode' (group) where OUT1_current or OUT2_current is linearly dependent on delta = IMEAS - X1.
# Strategy: scan single-column groupings, then two-column combinations if needed. Save top candidates' plots.
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from itertools import combinations

csv_path = Path(r"c:\Columbia\AdamResearch\kalman_2025\measurements\compute_20251219_123128.csv")
df = pd.read_csv(csv_path)
df['delta'] = df['IMEAS'] - df['X1']
# convert to nA
for col in ['OUT1_current', 'OUT2_current']:
    df[col + '_nA'] = df[col] * 1e9

lo_nA = 0.1
hi_nA = 100.0
min_points = 50  # minimum points in a group to consider

def fit_metrics(x, y):
    x = np.asarray(x); y = np.asarray(y)
    if len(x) < 3 or np.nanstd(x) == 0:
        return None
    slope, intercept = np.polyfit(x, y, 1)
    yhat = intercept + slope * x
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = 1 - ss_res/ss_tot if ss_tot > 0 else np.nan
    pearson_r = np.corrcoef(x, y)[0,1]
    return dict(slope=slope, intercept=intercept, r2=r2, pearson_r=pearson_r)

candidates = []
# choose candidate columns with reasonable cardinality
excluded = set(['IMEAS','X1','OUT1_current','OUT2_current','OUT1','OUT2','OUT1_current_nA','OUT2_current_nA','delta'])
col_nunique = {c: df[c].nunique(dropna=False) for c in df.columns if c not in excluded}
# Only consider columns with <= 100 unique values (modes)
mode_cols = [c for c,n in col_nunique.items() if n <= 100]
print('Candidate mode columns and unique counts:', {c:col_nunique[c] for c in mode_cols})

# Single-column scan
for col in mode_cols:
    for val, grp in df.groupby(col):
        for dep in ['OUT1_current','OUT2_current']:
            dep_n = dep + '_nA'
            sub = grp[(grp[dep_n] >= lo_nA) & (grp[dep_n] <= hi_nA) & grp['delta'].notnull()]
            n = len(sub)
            if n < min_points:
                continue
            m = fit_metrics(sub['delta'], sub[dep_n])
            if m is None:
                continue
            candidates.append(dict(group_cols=[col], group_vals=[val], dep=dep, n=n, **m))

# If no strong candidate found, do two-column combos (limit to columns with <=20 unique values to keep manageable)
strong_threshold = 0.2  # R^2 threshold to consider strong
candidates_sorted = sorted(candidates, key=lambda x: x['r2'] if not np.isnan(x['r2']) else -np.inf, reverse=True)
print('\nTop single-column candidates (by R2):')
for c in candidates_sorted[:10]:
    print(c)

if not any((c['r2'] >= strong_threshold) for c in candidates_sorted):
    print('\nNo single-column group crossed R^2 >=', strong_threshold, '- scanning two-column combos...')
    mode_cols_small = [c for c in mode_cols if col_nunique[c] <= 20]
    two_candidates = []
    for col1, col2 in combinations(mode_cols_small, 2):
        for vals, grp in df.groupby([col1, col2]):
            for dep in ['OUT1_current','OUT2_current']:
                dep_n = dep + '_nA'
                sub = grp[(grp[dep_n] >= lo_nA) & (grp[dep_n] <= hi_nA) & grp['delta'].notnull()]
                n = len(sub)
                if n < min_points:
                    continue
                m = fit_metrics(sub['delta'], sub[dep_n])
                if m is None:
                    continue
                two_candidates.append(dict(group_cols=[col1,col2], group_vals=list(vals), dep=dep, n=n, **m))
    two_sorted = sorted(two_candidates, key=lambda x: x['r2'] if not np.isnan(x['r2']) else -np.inf, reverse=True)
    print('\nTop two-column candidates (by R2):')
    for c in two_sorted[:10]:
        print(c)
    # merge into candidates_sorted for reporting
    candidates_sorted = candidates_sorted + two_sorted

# Take top 6 overall by R2
top = [c for c in sorted(candidates_sorted, key=lambda x: x['r2'] if not np.isnan(x['r2']) else -np.inf, reverse=True)][:6]
print('\nTop overall candidates:')
for i,c in enumerate(top,1):
    print(f"{i}. dep={c['dep']}, cols={c['group_cols']}, vals={c['group_vals']}, n={c['n']}, R2={c['r2']:.4f}, pearson={c['pearson_r']:.4f}")

# Save plots for these top candidates
plot_dir = Path('analysis_plots')
for i,c in enumerate(top,1):
    if c['n'] < min_points:
        continue
    # build mask
    mask = np.ones(len(df), dtype=bool)
    for col, val in zip(c['group_cols'], c['group_vals']):
        mask &= (df[col] == val)
    dep_n = c['dep'] + '_nA'
    sub = df[mask & (df[dep_n] >= lo_nA) & (df[dep_n] <= hi_nA) & df['delta'].notnull()]
    x = sub['delta'].to_numpy(); y = sub[dep_n].to_numpy()
    slope = c['slope']; intercept = c['intercept']
    xs = np.linspace(x.min(), x.max(), 200)
    ys = intercept + slope * xs
    plt.figure(figsize=(6,4))
    plt.scatter(x, y, s=8)
    plt.plot(xs, ys, color='C1')
    plt.xlabel('IMEAS - X1')
    plt.ylabel(f"{c['dep']} (nA)")
    title = f"{c['dep']} | {c['group_cols']}={c['group_vals']} (n={c['n']}) R2={c['r2']:.4f}"
    plt.title(title)
    plt.grid(alpha=0.3)
    p = plot_dir / f"candidate_{i}_{c['dep']}.png"
    plt.savefig(p, dpi=150, bbox_inches='tight')
    plt.close()
    c['plot'] = str(p)

print('\nPlots for top candidates saved to analysis_plots/ and candidate details:')
for c in top:
    print(c)

# Return top list for programmatic use
_top = top


Candidate mode columns and unique counts: {'PPG_state': 2, 'VDD': 1, 'VCC': 1, 'ERASE_PROG': 2, 'KGAIN1': 4, 'KGAIN2': 4, 'TRIM1': 4, 'TRIM2': 4, 'X2': 2, 'IREFP': 2, 'F11': 1, 'F12': 2}

Top single-column candidates (by R2):
{'group_cols': ['KGAIN1'], 'group_vals': [0.0], 'dep': 'OUT2_current', 'n': 28213, 'slope': np.float64(215432619.19596842), 'intercept': np.float64(11.079848616901872), 'r2': np.float64(0.01886798189338157), 'pearson_r': np.float64(0.13736077276057276)}
{'group_cols': ['KGAIN2'], 'group_vals': [0.0], 'dep': 'OUT2_current', 'n': 28213, 'slope': np.float64(215432619.19596842), 'intercept': np.float64(11.079848616901872), 'r2': np.float64(0.01886798189338157), 'pearson_r': np.float64(0.13736077276057276)}
{'group_cols': ['TRIM1'], 'group_vals': [5e-09], 'dep': 'OUT2_current', 'n': 28647, 'slope': np.float64(75438002.61810444), 'intercept': np.float64(8.553355801351001), 'r2': np.float64(0.006264250263298243), 'pearson_r': np.float64(0.07914701676815364)}
{'group_cols

In [3]:
# Print concise top candidate summary stored in _top
try:
    for i,c in enumerate(_top,1):
        print(f"{i}. dep={c['dep']}, cols={c['group_cols']}, vals={c['group_vals']}, n={c['n']}, R2={c['r2']:.4f}, pearson={c['pearson_r']:.4f}, plot={c.get('plot','n/a')}")
except Exception as e:
    print('No _top variable or error:', e)


1. dep=OUT2_current, cols=['PPG_state', 'IREFP'], vals=['ERASE', np.float64(1e-08)], n=71, R2=0.0831, pearson=-0.2883, plot=analysis_plots\candidate_1_OUT2_current.png
2. dep=OUT2_current, cols=['ERASE_PROG', 'IREFP'], vals=[np.float64(5.0), np.float64(1e-08)], n=71, R2=0.0831, pearson=-0.2883, plot=analysis_plots\candidate_2_OUT2_current.png
3. dep=OUT2_current, cols=['KGAIN1', 'IREFP'], vals=[np.float64(0.0), np.float64(1e-07)], n=13816, R2=0.0452, pearson=0.2126, plot=analysis_plots\candidate_3_OUT2_current.png
4. dep=OUT2_current, cols=['KGAIN2', 'IREFP'], vals=[np.float64(0.0), np.float64(1e-07)], n=13816, R2=0.0452, pearson=0.2126, plot=analysis_plots\candidate_4_OUT2_current.png
5. dep=OUT2_current, cols=['KGAIN1', 'TRIM1'], vals=[np.float64(0.0), np.float64(5e-08)], n=7040, R2=0.0268, pearson=0.1639, plot=analysis_plots\candidate_5_OUT2_current.png
6. dep=OUT2_current, cols=['KGAIN1', 'TRIM2'], vals=[np.float64(0.0), np.float64(5e-08)], n=7040, R2=0.0268, pearson=0.1639, plot=a

In [4]:
# Detailed regression for the top candidate: PPG_state='ERASE' and IREFP=1e-08, dependent OUT2_current
import numpy as np
import statsmodels.api as sm
from scipy import stats
from pathlib import Path

plot_dir = Path('analysis_plots')
col_filters = dict(PPG_state='ERASE', IREFP=np.float64(1e-08))
df_sub = df.copy()
for col, val in col_filters.items():
    df_sub = df_sub[df_sub[col] == val]

dep = 'OUT2_current'
dep_n = dep + '_nA'
sub = df_sub[(df_sub[dep_n] >= lo_nA) & (df_sub[dep_n] <= hi_nA) & df_sub['delta'].notnull()]
print('Group size after filtering:', len(sub))

x = sub['delta'].to_numpy(); y = sub[dep_n].to_numpy()
# Scipy linregress
lr = stats.linregress(x, y)
print('scipy linregress:', lr)
# statsmodels OLS
Xsm = sm.add_constant(x)
model = sm.OLS(y, Xsm).fit()
print(model.summary())

# Residuals and diagnostics
yhat = model.predict(Xsm)
resid = y - yhat

# Save scatter + fit
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
plt.scatter(x, y, s=12)
xs = np.linspace(x.min(), x.max(), 200)
plt.plot(xs, model.params[0] + model.params[1]*xs, color='C1')
plt.xlabel('IMEAS - X1')
plt.ylabel(f'{dep} (nA)')
plt.title(f"{dep} | PPG_state='ERASE', IREFP=1e-08 (n={len(sub)})\nR^2={model.rsquared:.4f}")
plt.grid(alpha=0.3)
p1 = plot_dir / 'top_candidate_scatter.png'
plt.savefig(p1, dpi=150, bbox_inches='tight')
plt.close()
print('Saved scatter to', p1)

# Residual plot
plt.figure(figsize=(6,3))
plt.scatter(yhat, resid, s=10)
plt.axhline(0, color='k', lw=0.7)
plt.xlabel('Fitted (nA)')
plt.ylabel('Residual (nA)')
plt.title('Residuals vs fitted')
p2 = plot_dir / 'top_candidate_resid.png'
plt.savefig(p2, dpi=150, bbox_inches='tight')
plt.close()
print('Saved residuals to', p2)

# Save results dict
top_detail = dict(n=len(sub), slope=lr.slope, intercept=lr.intercept, r2=lr.rvalue**2, pvalue=lr.pvalue, stderr=lr.stderr)
print('Top candidate stats:', top_detail)


Group size after filtering: 71
scipy linregress: LinregressResult(slope=np.float64(-1078242767.554653), intercept=np.float64(28.13983379825224), rvalue=np.float64(-0.28830421233978704), pvalue=np.float64(0.014761096784198536), stderr=np.float64(431119266.4671866), intercept_stderr=np.float64(7.631647075559603))
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     6.255
Date:                Mon, 29 Dec 2025   Prob (F-statistic):             0.0148
Time:                        13:32:17   Log-Likelihood:                -233.16
No. Observations:                  71   AIC:                             470.3
Df Residuals:                      69   BIC:                             474.8
Df Model:                           1                  